<a href="https://colab.research.google.com/github/KorStats/ocr_receipt/blob/main/%EC%A6%9D%EB%B6%84/O%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EC%A6%9D%EB%B6%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화  'Colab Notebooks'   images.zip        labels.zip   val.txt
 classes.txt	  images_1650.zip    labels_1650.zip  'My Drive'


In [3]:
!unzip /content/gdrive/'My Drive'/산업분류자동화/'실습용 자료'.zip -d /content/dataset

Archive:  /content/gdrive/My Drive/산업분류자동화/실습용 자료.zip
1. 실습용자료.txt:  mismatching "local" filename (1. ьЛдьК╡ьЪйьЮРыгМ.txt),
         continuing with "central" filename version
  inflating: /content/dataset/1. 실습용자료.txt  


In [4]:
import pandas as pd
df=pd.read_csv('/content/dataset/1. 실습용자료.txt', sep = "|", encoding = "euc-kr")

## 데이터 전처리

In [5]:
df=df[(df['digit_1'] =='O')] #조건에 맞는 데이터 출력

In [6]:
df=df.reset_index(drop=True)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000009,O,84,841,시청에서 재정과인력,일반공공행정,지방행정 집행
1,id_0000090,O,84,841,시청에서,지방행정사무를 집행,행정서비스
2,id_0001230,O,84,841,시청에서,지방행정 사무집행,행정지원서비스
3,id_0001352,O,84,841,가락동주민센터,지방행정사무를 집행,행정서비스
4,id_0002073,O,84,841,시청에서,지방행정사무,"행정사무집행, 민원서비스"
...,...,...,...,...,...,...,...
2960,id_0998659,O,84,841,시청에서,일반행정사무 집행,행정서비스
2961,id_0998832,O,84,842,사업장에서,산업행정,중소기업수출지원업무
2962,id_0998892,O,84,841,시청에서,일반행정,집행
2963,id_0998956,O,84,841,시청에서,일반 행정 집행기관,제천시청


In [7]:
df['text']=df['text_obj'].map(str)+" "+df['text_mthd'].map(str)+" "+df['text_deal'].map(str)
i=[]
for sentence in df['text']:
    a=sentence.replace('nan', '')
    i.append(a)
df['text']=pd.DataFrame(i)
df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000009,O,84,841,시청에서 재정과인력 일반공공행정 지방행정 집행
1,id_0000090,O,84,841,시청에서 지방행정사무를 집행 행정서비스
2,id_0001230,O,84,841,시청에서 지방행정 사무집행 행정지원서비스
3,id_0001352,O,84,841,가락동주민센터 지방행정사무를 집행 행정서비스
4,id_0002073,O,84,841,"시청에서 지방행정사무 행정사무집행, 민원서비스"
...,...,...,...,...,...
2960,id_0998659,O,84,841,시청에서 일반행정사무 집행 행정서비스
2961,id_0998832,O,84,842,사업장에서 산업행정 중소기업수출지원업무
2962,id_0998892,O,84,841,시청에서 일반행정 집행
2963,id_0998956,O,84,841,시청에서 일반 행정 집행기관 제천시청


## **# 데이터 증분 및 맞춤법 교정**

In [ ]:
# 맞춤법 검사

In [8]:
!pip install git+https://github.com/ssut/py-hanspell

  Cloning https://github.com/ssut/py-hanspell to /tmp/pip-req-build-if7lt5rp
  Running command git clone -q https://github.com/ssut/py-hanspell /tmp/pip-req-build-if7lt5rp
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=7b6ab6ff6ab574efad40b40eaca95568bf7797c4ca75d31dd255c2219a93098a
  Stored in directory: /tmp/pip-ephem-wheel-cache-8nl2vsbo/wheels/31/c2/c7/ddbbd9a84ddb0a67c2e52e3324d4b7b47dcf9581af9cd784de
Successfully built py-hanspell


In [9]:
from hanspell import spell_checker

temp_result=[]
for sentence in df['text']:
  try:
    result=spell_checker.check(sentence)
    temp_result.append(result.checked)
  except:
    temp_result.append(sentence)


df['text']=pd.DataFrame(temp_result)  

In [10]:
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000009,O,84,841,시청에서 재정과 인력 일반 공공행정 지방행정 집행
1,id_0000090,O,84,841,시청에서 지방행정사무를 집행 행정서비스
2,id_0001230,O,84,841,시청에서 지방행정 사무집행 행정지원 서비스
3,id_0001352,O,84,841,가락동 주민센터 지방행정사무를 집행 행정서비스
4,id_0002073,O,84,841,"시청에서 지방행정사무 행정사무집행, 민원서비스"
...,...,...,...,...,...
2960,id_0998659,O,84,841,시청에서 일반행정사무 집행 행정서비스
2961,id_0998832,O,84,842,사업장에서 산업 행정 중소기업 수출지원 업무
2962,id_0998892,O,84,841,시청에서 일반행정 집행
2963,id_0998956,O,84,841,시청에서 일반 행정 집행기관 제천시청


In [11]:
df['digit_3'].value_counts()

844    1267
841    1189
842     467
845      37
843       5
Name: digit_3, dtype: int64

In [12]:
# 데이터 증분 코드
import random
import pickle
import re
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)
	return parseText

def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break
	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")
	else:
		new_words = ""
	return new_words

def get_synonyms(word):
	synomyms = []
	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass
	return synomyms

def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)
	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]
	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)
	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)	
	return new_words

def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""
		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)

def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)
	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1
	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))
	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))
	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))
	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))
	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))
	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)
	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]
	augmented_sentences.append(sentence)
	return augmented_sentences

In [13]:
df1=df[df['digit_3']==841] 
df2=df[df['digit_3']==842] #1번 확정
df3=df[df['digit_3']==843] #251번 확정
df4=df[df['digit_3']==844] 
df5=df[df['digit_3']==845] #32번 확정

In [14]:
df1=df1.reset_index(drop=True)
df2=df2.reset_index(drop=True)
df3=df3.reset_index(drop=True)
df4=df4.reset_index(drop=True)
df5=df5.reset_index(drop=True)

In [15]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df2['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=1)
  id_list=['p'+str(df2['AI_id'][n]), 'p'+str(df2['AI_id'][n])]
  digit_1_list=[df2['digit_1'][n], df2['digit_1'][n]]
  digit_2_list=[df2['digit_2'][n], df2['digit_2'][n]]
  digit_3_list=[df2['digit_3'][n], df2['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df2_to_insert = pd.DataFrame(data_to_insert)
  df2= df2.append(df2_to_insert, ignore_index=True)
  n=n+1

In [16]:
df2

,AI_id,digit_1,digit_2,digit_3,text
0,id_0002146,O,84,842,정부 예산 공사 발주 등으로 도로관리 서비스 제공
1,id_0009324,O,84,842,시청 차량등록 사업소에서 오산시 관내 차량등록 및 관리 차량관리 행정서비스
2,id_0011799,O,84,842,어린이집의 신청을 받아 식단 및 위생 식자재 등 어린이 급식관리 지원
3,id_0014993,O,84,842,공공행정기관에서 도로교통법 교육 행정서비스
4,id_0019505,O,84,842,"공교육 기관으로 포천시 초, 중, 고 학교의 교육행정 재정 지원 및 관리"
...,...,...,...,...,...
1396,pid_0994196,O,84,842,사무실에서 관광시설물 운영 관리 관광행정
1397,pid_0997095,O,84,842,신항에서 수입식품검사 식품검사
1398,pid_0997095,O,84,842,신항에서 수입식품검사 식품검사
1399,pid_0998832,O,84,842,사업장에서 산업 행정 중소기업 수출지원 업무


In [17]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df5['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=32)
  id_list=['p'+str(df5['AI_id'][n]), 'p'+str(df5['AI_id'][n]), 'p'+str(df5['AI_id'][n]), 'p'+str(df5['AI_id'][n]),'p'+str(df5['AI_id'][n]), 'p'+str(df5['AI_id'][n]), 'p'+str(df5['AI_id'][n]), 'p'+str(df5['AI_id'][n]),'p'+str(df5['AI_id'][n]), 'p'+str(df5['AI_id'][n]), 'p'+str(df5['AI_id'][n]), 'p'+str(df5['AI_id'][n]),'p'+str(df5['AI_id'][n]), 'p'+str(df5['AI_id'][n]), 'p'+str(df5['AI_id'][n]), 'p'+str(df5['AI_id'][n]),'p'+str(df5['AI_id'][n]), 'p'+str(df5['AI_id'][n]), 'p'+str(df5['AI_id'][n]), 'p'+str(df5['AI_id'][n]),'p'+str(df5['AI_id'][n]), 'p'+str(df5['AI_id'][n]), 'p'+str(df5['AI_id'][n]), 'p'+str(df5['AI_id'][n]),'p'+str(df5['AI_id'][n]), 'p'+str(df5['AI_id'][n]), 'p'+str(df5['AI_id'][n]), 'p'+str(df5['AI_id'][n]),'p'+str(df5['AI_id'][n]), 'p'+str(df5['AI_id'][n]), 'p'+str(df5['AI_id'][n]), 'p'+str(df5['AI_id'][n]), 'p'+str(df5['AI_id'][n])]
  digit_1_list=[df5['digit_1'][n], df5['digit_1'][n], df5['digit_1'][n], df5['digit_1'][n],df5['digit_1'][n], df5['digit_1'][n], df5['digit_1'][n], df5['digit_1'][n],df5['digit_1'][n], df5['digit_1'][n], df5['digit_1'][n], df5['digit_1'][n],df5['digit_1'][n], df5['digit_1'][n], df5['digit_1'][n], df5['digit_1'][n],df5['digit_1'][n], df5['digit_1'][n], df5['digit_1'][n], df5['digit_1'][n],df5['digit_1'][n], df5['digit_1'][n], df5['digit_1'][n], df5['digit_1'][n],df5['digit_1'][n], df5['digit_1'][n], df5['digit_1'][n], df5['digit_1'][n],df5['digit_1'][n], df5['digit_1'][n], df5['digit_1'][n], df5['digit_1'][n],df5['digit_1'][n]]
  digit_2_list=[df5['digit_2'][n], df5['digit_2'][n], df5['digit_2'][n], df5['digit_2'][n],df5['digit_2'][n], df5['digit_2'][n], df5['digit_2'][n], df5['digit_2'][n],df5['digit_2'][n], df5['digit_2'][n], df5['digit_2'][n], df5['digit_2'][n],df5['digit_2'][n], df5['digit_2'][n], df5['digit_2'][n], df5['digit_2'][n],df5['digit_2'][n], df5['digit_2'][n], df5['digit_2'][n], df5['digit_2'][n],df5['digit_2'][n], df5['digit_2'][n], df5['digit_2'][n], df5['digit_2'][n],df5['digit_2'][n], df5['digit_2'][n], df5['digit_2'][n], df5['digit_2'][n],df5['digit_2'][n], df5['digit_2'][n], df5['digit_2'][n], df5['digit_2'][n],df5['digit_2'][n]]
  digit_3_list=[df5['digit_3'][n], df5['digit_3'][n], df5['digit_3'][n], df5['digit_3'][n],df5['digit_3'][n], df5['digit_3'][n], df5['digit_3'][n], df5['digit_3'][n],df5['digit_3'][n], df5['digit_3'][n], df5['digit_3'][n], df5['digit_3'][n],df5['digit_3'][n], df5['digit_3'][n], df5['digit_3'][n], df5['digit_3'][n],df5['digit_3'][n], df5['digit_3'][n], df5['digit_3'][n], df5['digit_3'][n],df5['digit_3'][n], df5['digit_3'][n], df5['digit_3'][n], df5['digit_3'][n],df5['digit_3'][n], df5['digit_3'][n], df5['digit_3'][n], df5['digit_3'][n],df5['digit_3'][n], df5['digit_3'][n], df5['digit_3'][n], df5['digit_3'][n],df5['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df5_to_insert = pd.DataFrame(data_to_insert)
  df5= df5.append(df5_to_insert, ignore_index=True)
  n=n+1

In [18]:
df5 #df1 완성함

,AI_id,digit_1,digit_2,digit_3,text
0,id_0086649,O,84,845,사무실에서 취업희망자에게 취업알선 및 직업교육
1,id_0126180,O,84,845,재단에서 복지. 행정 복지. 행정
2,id_0198561,O,84,845,사무실에서 사회보장 행정 실업 보상 업무 등 행정
3,id_0231558,O,84,845,센터에서 실업인을 대상으로 취업지원 등
4,id_0231677,O,84,845,"복지센터 일반 근로자 고용보험, 실업급여"
...,...,...,...,...,...
1253,pid_0956167,O,84,845,일자리 창녕 이익을 제공 공동의 사무실에서 추구하는
1254,pid_0956167,O,84,845,사무실에서 공동의 이익을 추구하는 창녕 일자리 제공
1255,pid_0956167,O,84,845,사무실에서 공동의 이익을 추구하는 창녕 일자리 제공
1256,pid_0956167,O,84,845,공동의 이익을 창녕 일자리 제공


In [19]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df3['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=251)
  id_list=['p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]),'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n]), 'p'+str(df3['AI_id'][n])]
  digit_1_list=[df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n], df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n],df3['digit_1'][n]]
  digit_2_list=[df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n], df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n],df3['digit_2'][n]]
  digit_3_list=[df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n], df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n],df3['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df3_to_insert = pd.DataFrame(data_to_insert)
  df3= df3.append(df3_to_insert, ignore_index=True)
  n=n+1

In [20]:
df3

,AI_id,digit_1,digit_2,digit_3,text
0,id_0246271,O,84,843,"병무청 모병, 징병 병무 행정서비스"
1,id_0246352,O,84,843,병무청에서 병무 행정서비스 병무 행정
2,id_0247360,O,84,843,병무청에서 병무 행정업무로 병역 관련 서비스
3,id_0247845,O,84,843,"병무청 육, 해, 공군, 민방위대 국가방위 수행"
4,id_0313320,O,84,843,"민방위 교육 민방위교육 훈련, 시민 생활안전체험 민방위교육. 관리, 시민 생활안전체험"
...,...,...,...,...,...
1260,pid_0313320,O,84,843,"민방위 교육 민방위교육 훈련, 시민 생활안전체험 민방위교육. 관리, 시민 생활안전체험"
1261,pid_0313320,O,84,843,민방위교육 시민 생활안전체험 민방위교육. 시민
1262,pid_0313320,O,84,843,"민방위 교육 민방위교육 훈련, 시민 생활안전체험 민방위교육. 관리, 시민 생활안전체험"
1263,pid_0313320,O,84,843,"민방위 교육 민방위교육 훈련, 시민 생활안전체험 민방위교육. 관리, 시민 생활안전체험"


기존 데이터 합치기

In [21]:
df=pd.concat([df1, df2,df3,df4,df5])

In [22]:
df['digit_3'].value_counts()

842    1401
844    1267
843    1265
845    1258
841    1189
Name: digit_3, dtype: int64

In [23]:
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000009,O,84,841,시청에서 재정과 인력 일반 공공행정 지방행정 집행
1,id_0000090,O,84,841,시청에서 지방행정사무를 집행 행정서비스
2,id_0001230,O,84,841,시청에서 지방행정 사무집행 행정지원 서비스
3,id_0001352,O,84,841,가락동 주민센터 지방행정사무를 집행 행정서비스
4,id_0002073,O,84,841,"시청에서 지방행정사무 행정사무집행, 민원서비스"
...,...,...,...,...,...
1253,pid_0956167,O,84,845,일자리 창녕 이익을 제공 공동의 사무실에서 추구하는
1254,pid_0956167,O,84,845,사무실에서 공동의 이익을 추구하는 창녕 일자리 제공
1255,pid_0956167,O,84,845,사무실에서 공동의 이익을 추구하는 창녕 일자리 제공
1256,pid_0956167,O,84,845,공동의 이익을 창녕 일자리 제공


In [24]:
df.to_csv('df_O_inc.csv', index=False, encoding='utf-8-sig')